In [2]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 46.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ef95b1db5689582e2ff02c43253c6059880a3694696e964461306272c249c05a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [134]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [135]:
from google.colab import files
uploads = files.upload()

Saving EmployeeSalary.csv to EmployeeSalary (1).csv


In [136]:
# creating a spark session
sc = SparkSession.builder.appName('sales').getOrCreate()

In [137]:
# reading the excel file
df = sc.read.csv('/content/EmployeeSalary.csv', header = True, inferSchema = True)
df.show(20)

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21|
|       ABS|Alcohol Beverage ...|      Administration|     F|   78902.0|     205.16|     2460.24|   18|
|       ABS|Alcohol Beverage ...|           Marketing|     F|  109761.0|        0.0|         0.0|   25|
|       ABS|Alcohol Beverage ...|Beer Delivery Ope...|     M|   

In [138]:
df.createOrReplaceTempView('dataframe')
size = sc.sql("SELECT COUNT(*) FROM dataframe").show()

+--------+
|count(1)|
+--------+
|    9958|
+--------+



In [139]:
df.printSchema()

root
 |-- Department: string (nullable = true)
 |-- DepartmentName: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- BaseSalary: double (nullable = true)
 |-- OvertimePay: double (nullable = true)
 |-- LongevityPay: double (nullable = true)
 |-- Grade: string (nullable = true)



In [140]:
df.columns

['Department',
 'DepartmentName',
 'Division',
 'Gender',
 'BaseSalary',
 'OvertimePay',
 'LongevityPay',
 'Grade']

In [141]:
df_Columns=['Department',
 'DepartmentName',
 'Division',
 'Gender',
 'BaseSalary',
 'OvertimePay',
 'LongevityPay',
 'Grade']
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]).show()

+----------+--------------+--------+------+----------+-----------+------------+-----+
|Department|DepartmentName|Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|
+----------+--------------+--------+------+----------+-----------+------------+-----+
|         0|             0|       0|     0|         0|          0|           0|    0|
+----------+--------------+--------+------+----------+-----------+------------+-----+



In [142]:
df = df.na.drop('any')

In [143]:
# adding a new col as MaxSalary with over partion of the department
res = sc.sql(
    "SELECT d.*,\
    max(BaseSalary) OVER(PARTITION BY DepartmentName) as MaxSalary\
    FROM dataframe d\
    "
).show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|MaxSalary|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18| 220000.0|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16| 220000.0|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2| 220000.0|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21| 220000.0|
|       ABS|Alcohol Beverage ...|      Administration|     F|   78902.0|     205.16|     2460.24|   18| 220000.0|
|       ABS|Alcohol Beverage ...|           Marketing|     F|  109761.0|        0.0|    

In [144]:
# adding row_number which is partitioned by department

row = sc.sql("SELECT d.*,\
            row_number() OVER(PARTITION BY DEPARTMENT ORDER BY DEPARTMENT) AS rowID\
            FROM dataframe d").show(10)

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+-----+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|rowID|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+-----+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|    1|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|    2|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|    3|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21|    4|
|       ABS|Alcohol Beverage ...|      Administration|     F|   78902.0|     205.16|     2460.24|   18|    5|
|       ABS|Alcohol Beverage ...|           Marketing|     F|  109761.0|        0.0|         0.0|   25|    6|
|       AB

In [145]:
# extracting first two employee from each department 
row = sc.sql("SELECT * FROM \
                (SELECT d.*,\
                    row_number()\
                    OVER(PARTITION BY Department ORDER BY Department) AS rowID\
                    FROM dataframe d ) x \
                WHERE x.rowID < 3").show(10)

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+-----+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|rowID|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+-----+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|    1|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|    2|
|       BOA|Board of Appeals ...|Board of Appeals ...|     F|   78902.0|        0.0|         0.0|   18|    1|
|       BOA|Board of Appeals ...|Board of Appeals ...|     F|  58481.91|        0.0|         0.0|   16|    2|
|       BOE|  Board of Elections|            Director|     F| 183654.18|        0.0|         0.0|   M1|    1|
|       BOE|  Board of Elections|Registration Serv...|     F|   62488.0|    5622.07|         0.0|   13|    2|
|       CA

In [146]:
# employee from each department earing  salary ordered using rank function
# for every duplicate record with same rank it will skip a rank eg: rank : 1,2,2,4 : this means there are two employee with rank 2, so it skips rank 3 and  goes to rank 4
rank = sc.sql("SELECT d.*,\
                rank() \
                OVER(PARTITION BY Department ORDER BY BaseSalary DESC )as rnk\
            FROM dataframe d").show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|rnk|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+
|       ABS|Alcohol Beverage ...|            Director|     M|  220000.0|        0.0|         0.0|    0|  1|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  2|
|       ABS|Alcohol Beverage ...|Licensure, Regula...|     F|  167345.0|        0.0|         0.0|   M2|  2|
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     M|  167345.0|        0.0|         0.0|   M2|  2|
|       ABS|Alcohol Beverage ...|           Marketing|     F| 164193.05|        0.0|         0.0|   M2|  5|
|       ABS|Alcohol Beverage ...|Retail Administra...|     M|  156442.5|        0.0|         0.0|   M2|  6|
|       ABS|Alcohol Beverage

In [147]:
# top3 exployee with highest salary partitioned by department 
rank_3 = sc.sql("SELECT * FROM (SELECT d.*,\
                rank() \
                OVER(PARTITION BY Department ORDER BY BaseSalary DESC )as rnk\
            FROM dataframe d) x WHERE x.rnk < 4 ").show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|rnk|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+
|       ABS|Alcohol Beverage ...|            Director|     M|  220000.0|        0.0|         0.0|    0|  1|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  2|
|       ABS|Alcohol Beverage ...|Licensure, Regula...|     F|  167345.0|        0.0|         0.0|   M2|  2|
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     M|  167345.0|        0.0|         0.0|   M2|  2|
|       BOA|Board of Appeals ...|  Executive Director|     F|  144751.0|        0.0|         0.0|   M3|  1|
|       BOA|Board of Appeals ...|Board of Appeals ...|     F|   78902.0|        0.0|         0.0|   18|  2|
|       BOA|Board of Appeals

In [148]:
# dense rank and row number

dns_rank = sc.sql(
    "SELECT d.*,\
        rank() OVER(PARTITION BY Department ORDER BY BaseSalary DESC) as rnk,\
        dense_rank() OVER(PARTITION BY Department ORDER BY BaseSalary DESC) as DENSE_rnk,\
        row_number() OVER(PARTITION BY Department ORDER BY BaseSalary DESC) as rn\
    FROM dataframe d").show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+---------+---+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|rnk|DENSE_rnk| rn|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---+---------+---+
|       ABS|Alcohol Beverage ...|            Director|     M|  220000.0|        0.0|         0.0|    0|  1|        1|  1|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  2|        2|  2|
|       ABS|Alcohol Beverage ...|Licensure, Regula...|     F|  167345.0|        0.0|         0.0|   M2|  2|        2|  3|
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     M|  167345.0|        0.0|         0.0|   M2|  2|        2|  4|
|       ABS|Alcohol Beverage ...|           Marketing|     F| 164193.05|        0.0|         0.0|   M2|  5|        3|  5|
|       ABS|Alcohol Beve

In [149]:
# LEAD and LAG function: Suppose if you want to check if the salary to the curretn employ is lower or greater than the pervious employee
# LAG(col, 2, 0): generates a lag column by using the info from specified col in the function and parameter '2', is it looks two steps behind and fills 0 to the lagged space.
ld = sc.sql(
    "SELECT d.*,\
    lAG(BaseSalary, 2,0) OVER(PARTITION BY Department ORDER BY Department) as preSalary\
    FROM dataframe d").show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|preSalary|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|      0.0|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|      0.0|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  78902.0|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21|  35926.0|
|       ABS|Alcohol Beverage ...|      Administration|     F|   78902.0|     205.16|     2460.24|   18| 167345.0|
|       ABS|Alcohol Beverage ...|           Marketing|     F|  109761.0|        0.0|    

In [150]:
# LEAD function
ld = sc.sql(
    "SELECT d.*,\
    lAG(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) as preSalary,\
    lEAD(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) as nxtSalary\
    FROM dataframe d").show()


+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+---------+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|preSalary|nxtSalary|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+---------+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|     null|  35926.0|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|  78902.0| 167345.0|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  35926.0|  90848.0|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21| 167345.0|  78902.0|
|       ABS|Alcohol Beverage ...|      Administration|     F|   78902.0|     205.16|     2460.24|   18|  90848.0| 109761.0|
|       

In [154]:
# fetch a query to disply if the salary of the employee is higher or lower than previous employee
ld = sc.sql(
    "SELECT d.*,\
    lAG(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) as preSalary,\
    CASE WHEN d.BaseSalary > LAG(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) then 'Higher than Previous Employee'\
        WHEN d.BaseSalary < LAG(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) then 'Lower than Previous Employee'\
        WHEN d.BaseSalary = LAG(BaseSalary) OVER(PARTITION BY Department ORDER BY Department) then 'Same as the Previous Employee'\
        END sal_range\
    FROM dataframe d").show()

+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+--------------------+
|Department|      DepartmentName|            Division|Gender|BaseSalary|OvertimePay|LongevityPay|Grade|preSalary|           sal_range|
+----------+--------------------+--------------------+------+----------+-----------+------------+-----+---------+--------------------+
|       ABS|Alcohol Beverage ...|Wholesale Adminis...|     F|   78902.0|     199.17|         0.0|   18|     null|                null|
|       ABS|Alcohol Beverage ...|Administrative Se...|     F|   35926.0|        0.0|     4038.91|   16|  78902.0|Lower than Previo...|
|       ABS|Alcohol Beverage ...|      Administration|     M|  167345.0|        0.0|         0.0|   M2|  35926.0|Higher than Previ...|
|       ABS|Alcohol Beverage ...|Wholesale Operations|     F|   90848.0|        0.0|     5717.68|   21| 167345.0|Lower than Previo...|
|       ABS|Alcohol Beverage ...|      Administration| 

In [159]:
# https://www.youtube.com/watch?v=zAmJPdZu8Rg


# creating a dataframe 
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


In [160]:
data = [('Phone', 'Apple', 'iPhone 12 Pro Max', 1300),
('Phone', 'Apple', 'iPhone 12 Pro', 1100),
('Phone', 'Apple', 'iPhone 12', 1000),
('Phone', 'Samsung', 'Galaxy Z Fold 3', 1800),
('Phone', 'Samsung', 'Galaxy Z Flip 3', 1000),
('Phone', 'Samsung', 'Galaxy Note 20', 1200),
('Phone', 'Samsung', 'Galaxy S21', 1000),
('Phone', 'OnePlus', 'OnePlus Nord', 300),
('Phone', 'OnePlus', 'OnePlus 9', 800),
('Phone', 'Google', 'Pixel 5', 600),
('Laptop', 'Apple', 'MacBook Pro 13', 2000),
('Laptop', 'Apple', 'MacBook Air', 1200),
('Laptop', 'Microsoft', 'Surface Laptop 4', 2100),
('Laptop', 'Dell', 'XPS 13', 2000),
('Laptop', 'Dell', 'XPS 15', 2300),
('Laptop', 'Dell', 'XPS 17', 2500),
('Earphone', 'Apple', 'AirPods Pro', 280),
('Earphone', 'Samsung', 'Galaxy Buds Pro', 220),
('Earphone', 'Samsung', 'Galaxy Buds Live', 170),
('Earphone', 'Sony', 'WF-1000XM4', 250),
('Headphone', 'Sony', 'WH-1000XM4', 400),
('Headphone', 'Apple', 'AirPods Max', 550),
('Headphone', 'Microsoft', 'Surface Headphones 2', 250),
('Smartwatch', 'Apple', 'Apple Watch Series 6', 1000),
('Smartwatch', 'Apple', 'Apple Watch SE', 400),
('Smartwatch', 'Samsung', 'Galaxy Watch 4', 600),
('Smartwatch', 'OnePlus', 'OnePlus Watch', 220)]

In [162]:
schema = StructType([ \
    StructField("ProductCat",StringType(),True), \
    StructField("Brand",StringType(),True), \
    StructField("ProductName",StringType(),True), \
    StructField("Price", IntegerType(), True) \
  ])

In [163]:
df = sc.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- ProductCat: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Price: integer (nullable = true)

+----------+---------+-----------------+-----+
|ProductCat|Brand    |ProductName      |Price|
+----------+---------+-----------------+-----+
|Phone     |Apple    |iPhone 12 Pro Max|1300 |
|Phone     |Apple    |iPhone 12 Pro    |1100 |
|Phone     |Apple    |iPhone 12        |1000 |
|Phone     |Samsung  |Galaxy Z Fold 3  |1800 |
|Phone     |Samsung  |Galaxy Z Flip 3  |1000 |
|Phone     |Samsung  |Galaxy Note 20   |1200 |
|Phone     |Samsung  |Galaxy S21       |1000 |
|Phone     |OnePlus  |OnePlus Nord     |300  |
|Phone     |OnePlus  |OnePlus 9        |800  |
|Phone     |Google   |Pixel 5          |600  |
|Laptop    |Apple    |MacBook Pro 13   |2000 |
|Laptop    |Apple    |MacBook Air      |1200 |
|Laptop    |Microsoft|Surface Laptop 4 |2100 |
|Laptop    |Dell     |XPS 13           |2000 |
|Laptop    |Dell     |XPS 15       

In [167]:
df.createOrReplaceTempView('dataset')
sc.sql("SELECT * FROM dataset").show()

+----------+---------+-----------------+-----+
|ProductCat|    Brand|      ProductName|Price|
+----------+---------+-----------------+-----+
|     Phone|    Apple|iPhone 12 Pro Max| 1300|
|     Phone|    Apple|    iPhone 12 Pro| 1100|
|     Phone|    Apple|        iPhone 12| 1000|
|     Phone|  Samsung|  Galaxy Z Fold 3| 1800|
|     Phone|  Samsung|  Galaxy Z Flip 3| 1000|
|     Phone|  Samsung|   Galaxy Note 20| 1200|
|     Phone|  Samsung|       Galaxy S21| 1000|
|     Phone|  OnePlus|     OnePlus Nord|  300|
|     Phone|  OnePlus|        OnePlus 9|  800|
|     Phone|   Google|          Pixel 5|  600|
|    Laptop|    Apple|   MacBook Pro 13| 2000|
|    Laptop|    Apple|      MacBook Air| 1200|
|    Laptop|Microsoft| Surface Laptop 4| 2100|
|    Laptop|     Dell|           XPS 13| 2000|
|    Laptop|     Dell|           XPS 15| 2300|
|    Laptop|     Dell|           XPS 17| 2500|
|  Earphone|    Apple|      AirPods Pro|  280|
|  Earphone|  Samsung|  Galaxy Buds Pro|  220|
|  Earphone| 

In [169]:
# first value: using it to query out the most expensive product from each category

fval = sc.sql(
    "SELECT *,\
    first_value(ProductName) OVER(PARTITION BY ProductCat ORDER BY price DESC) as mostExpensiveProduct\
     FROM dataset"
).show()

+----------+---------+--------------------+-----+--------------------+
|ProductCat|    Brand|         ProductName|Price|mostExpensiveProduct|
+----------+---------+--------------------+-----+--------------------+
|  Earphone|    Apple|         AirPods Pro|  280|         AirPods Pro|
|  Earphone|     Sony|          WF-1000XM4|  250|         AirPods Pro|
|  Earphone|  Samsung|     Galaxy Buds Pro|  220|         AirPods Pro|
|  Earphone|  Samsung|    Galaxy Buds Live|  170|         AirPods Pro|
| Headphone|    Apple|         AirPods Max|  550|         AirPods Max|
| Headphone|     Sony|          WH-1000XM4|  400|         AirPods Max|
| Headphone|Microsoft|Surface Headphones 2|  250|         AirPods Max|
|    Laptop|     Dell|              XPS 17| 2500|              XPS 17|
|    Laptop|     Dell|              XPS 15| 2300|              XPS 17|
|    Laptop|Microsoft|    Surface Laptop 4| 2100|              XPS 17|
|    Laptop|    Apple|      MacBook Pro 13| 2000|              XPS 17|
|    L

In [179]:
lstval = sc.sql(
    "SELECT *,\
    FIRST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY price DESC)\
        AS mostExpensiveProduct,\
    LAST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY Price DESC)\
        AS leastExpensiveProduct\
    FROM dataset"
).show()

+----------+---------+--------------------+-----+--------------------+---------------------+
|ProductCat|    Brand|         ProductName|Price|mostExpensiveProduct|leastExpensiveProduct|
+----------+---------+--------------------+-----+--------------------+---------------------+
|  Earphone|    Apple|         AirPods Pro|  280|         AirPods Pro|          AirPods Pro|
|  Earphone|     Sony|          WF-1000XM4|  250|         AirPods Pro|           WF-1000XM4|
|  Earphone|  Samsung|     Galaxy Buds Pro|  220|         AirPods Pro|      Galaxy Buds Pro|
|  Earphone|  Samsung|    Galaxy Buds Live|  170|         AirPods Pro|     Galaxy Buds Live|
| Headphone|    Apple|         AirPods Max|  550|         AirPods Max|          AirPods Max|
| Headphone|     Sony|          WH-1000XM4|  400|         AirPods Max|           WH-1000XM4|
| Headphone|Microsoft|Surface Headphones 2|  250|         AirPods Max| Surface Headphones 2|
|    Laptop|     Dell|              XPS 17| 2500|              XPS 17|

In [182]:
# lastvalue: to query out the least expensive product from each product cat 
# 'RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW' is the default frame claus in SQL ( dont have to mention, its just for refrence)
#'RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING' has to be mention when using last_value(), nth value and some agg window function (check above for refrence to see the output without frameclaus)
lstval = sc.sql(
    "SELECT *,\
    FIRST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY price DESC)\
        AS mostExpensiveProduct,\
    LAST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY Price DESC\
        RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)\
        AS leastExpensiveProduct\
    FROM dataset"
).show()

+----------+---------+--------------------+-----+--------------------+---------------------+
|ProductCat|    Brand|         ProductName|Price|mostExpensiveProduct|leastExpensiveProduct|
+----------+---------+--------------------+-----+--------------------+---------------------+
|  Earphone|    Apple|         AirPods Pro|  280|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|     Sony|          WF-1000XM4|  250|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|  Samsung|     Galaxy Buds Pro|  220|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|  Samsung|    Galaxy Buds Live|  170|         AirPods Pro|     Galaxy Buds Live|
| Headphone|    Apple|         AirPods Max|  550|         AirPods Max| Surface Headphones 2|
| Headphone|     Sony|          WH-1000XM4|  400|         AirPods Max| Surface Headphones 2|
| Headphone|Microsoft|Surface Headphones 2|  250|         AirPods Max| Surface Headphones 2|
|    Laptop|     Dell|              XPS 17| 2500|              XPS 17|

In [187]:
# extracting least and most expensive phone and laptop

phone = sc.sql(
    "SELECT *,\
    FIRST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY price DESC)\
        AS mostExpensiveProduct,\
    LAST_VALUE(ProductName) \
        OVER(PARTITION BY ProductCat ORDER BY Price DESC\
        RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)\
        AS leastExpensiveProduct\
    FROM dataset\
    WHERE (ProductCat = 'Phone' OR ProductCat = 'Laptop')"
).show()

+----------+---------+-----------------+-----+--------------------+---------------------+
|ProductCat|    Brand|      ProductName|Price|mostExpensiveProduct|leastExpensiveProduct|
+----------+---------+-----------------+-----+--------------------+---------------------+
|    Laptop|     Dell|           XPS 17| 2500|              XPS 17|          MacBook Air|
|    Laptop|     Dell|           XPS 15| 2300|              XPS 17|          MacBook Air|
|    Laptop|Microsoft| Surface Laptop 4| 2100|              XPS 17|          MacBook Air|
|    Laptop|    Apple|   MacBook Pro 13| 2000|              XPS 17|          MacBook Air|
|    Laptop|     Dell|           XPS 13| 2000|              XPS 17|          MacBook Air|
|    Laptop|    Apple|      MacBook Air| 1200|              XPS 17|          MacBook Air|
|     Phone|  Samsung|  Galaxy Z Fold 3| 1800|     Galaxy Z Fold 3|         OnePlus Nord|
|     Phone|    Apple|iPhone 12 Pro Max| 1300|     Galaxy Z Fold 3|         OnePlus Nord|
|     Phon

In [189]:
# different way to write a window function

phone = sc.sql(
    "SELECT *,\
    FIRST_VALUE(ProductName) OVER w AS mostExpensiveProduct,\
    LAST_VALUE(ProductName) OVER w AS leastExpensiveProduct\
    FROM dataset\
    WINDOW w as (PARTITION BY ProductCat ORDER BY Price DESC\
        RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)"
).show()

+----------+---------+--------------------+-----+--------------------+---------------------+
|ProductCat|    Brand|         ProductName|Price|mostExpensiveProduct|leastExpensiveProduct|
+----------+---------+--------------------+-----+--------------------+---------------------+
|  Earphone|    Apple|         AirPods Pro|  280|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|     Sony|          WF-1000XM4|  250|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|  Samsung|     Galaxy Buds Pro|  220|         AirPods Pro|     Galaxy Buds Live|
|  Earphone|  Samsung|    Galaxy Buds Live|  170|         AirPods Pro|     Galaxy Buds Live|
| Headphone|    Apple|         AirPods Max|  550|         AirPods Max| Surface Headphones 2|
| Headphone|     Sony|          WH-1000XM4|  400|         AirPods Max| Surface Headphones 2|
| Headphone|Microsoft|Surface Headphones 2|  250|         AirPods Max| Surface Headphones 2|
|    Laptop|     Dell|              XPS 17| 2500|              XPS 17|

In [191]:
# Nth value: neth_value(col, nthval(int)): nth val require two input arguments, 
#1st is the col from which you want to represent the nth vaöu and the second argument will be woth value eg: 1st , 2nd or..

# write a query to disply 2nd most expensive product under each category

nthval = sc.sql(
    "SELECT *,\
    FIRST_VALUE(ProductName) OVER w AS mostExpensiveProduct,\
    LAST_VALUE(ProductName) OVER w AS leastExpensiveProduct,\
    nth_value(ProductName, 2) OVER w AS secondMostExpensive\
    FROM dataset\
    WINDOW w as (PARTITION BY ProductCat ORDER BY Price DESC\
        RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)"
).show()

+----------+-----+--------------------+-----+--------------------+---------------------+-------------------+
|ProductCat|Brand|         ProductName|Price|mostExpensiveProduct|leastExpensiveProduct|secondMostExpensive|
+----------+-----+--------------------+-----+--------------------+---------------------+-------------------+
|  Earphone|Apple|         AirPods Pro|  280|         AirPods Pro|          AirPods Pro|               null|
| Headphone|Apple|         AirPods Max|  550|         AirPods Max|          AirPods Max|               null|
|    Laptop|Apple|      MacBook Pro 13| 2000|      MacBook Pro 13|          MacBook Air|        MacBook Air|
|    Laptop|Apple|         MacBook Air| 1200|      MacBook Pro 13|          MacBook Air|        MacBook Air|
|     Phone|Apple|   iPhone 12 Pro Max| 1300|   iPhone 12 Pro Max|            iPhone 12|      iPhone 12 Pro|
|     Phone|Apple|       iPhone 12 Pro| 1100|   iPhone 12 Pro Max|            iPhone 12|      iPhone 12 Pro|
|     Phone|Apple| 

In [199]:
# NTILE : it splits equally
# write a query to segregate all the expensive phone, mid range phone and cheaper phones

ntile = sc.sql(
    "SELECT *,\
        ntile(3) OVER(ORDER BY Price DESC) AS buckets\
        FROM dataset\
    WHERE ProductCat = 'Phone'").show()

+----------+-------+-----------------+-----+-------+
|ProductCat|  Brand|      ProductName|Price|buckets|
+----------+-------+-----------------+-----+-------+
|     Phone|Samsung|  Galaxy Z Fold 3| 1800|      1|
|     Phone|  Apple|iPhone 12 Pro Max| 1300|      1|
|     Phone|Samsung|   Galaxy Note 20| 1200|      1|
|     Phone|  Apple|    iPhone 12 Pro| 1100|      1|
|     Phone|  Apple|        iPhone 12| 1000|      2|
|     Phone|Samsung|  Galaxy Z Flip 3| 1000|      2|
|     Phone|Samsung|       Galaxy S21| 1000|      2|
|     Phone|OnePlus|        OnePlus 9|  800|      3|
|     Phone| Google|          Pixel 5|  600|      3|
|     Phone|OnePlus|     OnePlus Nord|  300|      3|
+----------+-------+-----------------+-----+-------+



In [201]:
# lets take the above query and assign string to each bucket val
ntile = sc.sql(
    "SELECT ProductName,Brand,\
    CASE\
    WHEN x.buckets = 1 THEN 'Expensive Phone'\
    WHEN x.buckets = 2 THEN 'Mid Range Phone'\
    WHEN x.buckets = 3 THEN 'Cheaper Phone'\
    END Phone_Category\
    FROM(SELECT *,\
        ntile(3) OVER(ORDER BY Price DESC) AS buckets\
        FROM dataset\
    WHERE ProductCat = 'Phone') x ").show()

+-----------------+-------+---------------+
|      ProductName|  Brand| Phone_Category|
+-----------------+-------+---------------+
|  Galaxy Z Fold 3|Samsung|Expensive Phone|
|iPhone 12 Pro Max|  Apple|Expensive Phone|
|   Galaxy Note 20|Samsung|Expensive Phone|
|    iPhone 12 Pro|  Apple|Expensive Phone|
|        iPhone 12|  Apple|Mid Range Phone|
|  Galaxy Z Flip 3|Samsung|Mid Range Phone|
|       Galaxy S21|Samsung|Mid Range Phone|
|        OnePlus 9|OnePlus|  Cheaper Phone|
|          Pixel 5| Google|  Cheaper Phone|
|     OnePlus Nord|OnePlus|  Cheaper Phone|
+-----------------+-------+---------------+



In [202]:
# cumulative distribution

cumdist = sc.sql(
    "SELECT *,\
    CUME_DIST() OVER(ORDER BY Price DESC) as cume_distribution\
    FROM dataset"
).show()

+----------+---------+--------------------+-----+--------------------+
|ProductCat|    Brand|         ProductName|Price|   cume_distribution|
+----------+---------+--------------------+-----+--------------------+
|    Laptop|     Dell|              XPS 17| 2500|0.037037037037037035|
|    Laptop|     Dell|              XPS 15| 2300| 0.07407407407407407|
|    Laptop|Microsoft|    Surface Laptop 4| 2100|  0.1111111111111111|
|    Laptop|    Apple|      MacBook Pro 13| 2000| 0.18518518518518517|
|    Laptop|     Dell|              XPS 13| 2000| 0.18518518518518517|
|     Phone|  Samsung|     Galaxy Z Fold 3| 1800|  0.2222222222222222|
|     Phone|    Apple|   iPhone 12 Pro Max| 1300| 0.25925925925925924|
|     Phone|  Samsung|      Galaxy Note 20| 1200|  0.3333333333333333|
|    Laptop|    Apple|         MacBook Air| 1200|  0.3333333333333333|
|     Phone|    Apple|       iPhone 12 Pro| 1100| 0.37037037037037035|
|     Phone|    Apple|           iPhone 12| 1000|  0.5185185185185185|
|     

In [209]:
# cumulative distribution and converting them to percentage for better visual

cumdist = sc.sql(
    "SELECT *,\
    ROUND(CUME_DIST() OVER(ORDER BY Price DESC),3) as cume_distribution,\
    ROUND(CUME_DIST() OVER(ORDER BY Price DESC) *100, 2) as cume_dist_percentage\
    FROM dataset").show()

+----------+---------+--------------------+-----+-----------------+--------------------+
|ProductCat|    Brand|         ProductName|Price|cume_distribution|cume_dist_percentage|
+----------+---------+--------------------+-----+-----------------+--------------------+
|    Laptop|     Dell|              XPS 17| 2500|            0.037|                 3.7|
|    Laptop|     Dell|              XPS 15| 2300|            0.074|                7.41|
|    Laptop|Microsoft|    Surface Laptop 4| 2100|            0.111|               11.11|
|    Laptop|    Apple|      MacBook Pro 13| 2000|            0.185|               18.52|
|    Laptop|     Dell|              XPS 13| 2000|            0.185|               18.52|
|     Phone|  Samsung|     Galaxy Z Fold 3| 1800|            0.222|               22.22|
|     Phone|    Apple|   iPhone 12 Pro Max| 1300|            0.259|               25.93|
|     Phone|  Samsung|      Galaxy Note 20| 1200|            0.333|               33.33|
|    Laptop|    Apple

In [214]:
# cumulative distribution and converting them to percentage and extract the top 30% of the data

cumdist = sc.sql("\
    SELECT ProductName, (cume_dist_percentage || '%') AS cume_dist_percentage\
        FROM\
            (SELECT *,\
                ROUND(CUME_DIST() OVER(ORDER BY Price DESC),3) as cume_distribution,\
                ROUND(CUME_DIST() OVER(ORDER BY Price DESC) *100, 2) as cume_dist_percentage\
            FROM dataset) x\
        WHERE x.cume_dist_percentage <= 30").show()

+-----------------+--------------------+
|      ProductName|cume_dist_percentage|
+-----------------+--------------------+
|           XPS 17|                3.7%|
|           XPS 15|               7.41%|
| Surface Laptop 4|              11.11%|
|   MacBook Pro 13|              18.52%|
|           XPS 13|              18.52%|
|  Galaxy Z Fold 3|              22.22%|
|iPhone 12 Pro Max|              25.93%|
+-----------------+--------------------+



In [219]:
# percent_rank : provides relative rank for each row
# how expensive is "Galaxy Z Fold 3" when compared to all products

gal = sc.sql(
    "SELECT *,\
        PERCENT_RANK() OVER(ORDER BY Price ASC) as percentRank,\
        ROUND(PERCENT_RANK() OVER(ORDER BY Price ASC)*100,2) as percentRank\
    FROM dataset").show()


+----------+---------+--------------------+-----+--------------------+-----------+
|ProductCat|    Brand|         ProductName|Price|         percentRank|percentRank|
+----------+---------+--------------------+-----+--------------------+-----------+
|  Earphone|  Samsung|    Galaxy Buds Live|  170|                 0.0|        0.0|
|  Earphone|  Samsung|     Galaxy Buds Pro|  220|0.038461538461538464|       3.85|
|Smartwatch|  OnePlus|       OnePlus Watch|  220|0.038461538461538464|       3.85|
|  Earphone|     Sony|          WF-1000XM4|  250| 0.11538461538461539|      11.54|
| Headphone|Microsoft|Surface Headphones 2|  250| 0.11538461538461539|      11.54|
|  Earphone|    Apple|         AirPods Pro|  280| 0.19230769230769232|      19.23|
|     Phone|  OnePlus|        OnePlus Nord|  300| 0.23076923076923078|      23.08|
| Headphone|     Sony|          WH-1000XM4|  400|  0.2692307692307692|      26.92|
|Smartwatch|    Apple|      Apple Watch SE|  400|  0.2692307692307692|      26.92|
| He